<a href="https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/Cuaderno_completo_So_Vits_SVC_en_espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="pink" size="+2"> Cuaderno completo de So-Vits SVC en español. (Entrenamiento + Inferencia)
---
## Siga los pasos, ejecutando las celdas en orden, para utilizar.
---
###Créditos:

* Cuaderno elaborado por: [rmcpantoja](https://github.com/rmcpantoja) y [Xx_Nessu_xX](https://fakeyou.com/profile/Xx_Nessu_xX)
* Cuaderno usado de base por: [IA HISPANO](https://colab.research.google.com/drive/1tR10bWZmQOTrXBnbOW-Ir84h66o2b633#scrollTo=g7o4_4zL5pAs)
* Cuaderno usado de base por: [Justin John](https://colab.research.google.com/github/justinjohn0306/so-vits-svc-4.0-v2/blob/main/sovits4v2_training_testing.ipynb#scrollTo=wmUkpUmfn_Hs)
* Repositorio de GitHub de [SoVits-SVC](https://github.com/justinjohn0306/so-vits-svc-4.0-v2)

###Agradecimientos:

* Testeadores del cuaderno: [Exink14](https://fakeyou.com/profile/exinkvoices1410), [Jawi95_](https://www.youtube.com/@jawi95_) y `Eduardo Petrini`.


# <font color="pink"> 🔧 ***Preparación del proyecto.*** 🔧

Puedes ejecutar toda esta sección tanto para el entrenamiento como para la inferencia, en el caso de que estés ejecutando este cuaderno por primera vez.

In [ ]:
#@markdown # <font color="pink"> **Google Colab Anti-Disconnect.** 🔌
#@markdown ---
#@markdown #### Evita la desconexión automática. Aun así, se desconectará después de <font color="orange">**6 a 12 horas**</font>.

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

In [ ]:
#@markdown # <font color="pink"> **Comprobar GPU asignada.** 👁️
!nvidia-smi
#@markdown ---
#@markdown #### Una GPU con mayor capacidad puede llevar a mayor velocidad de entrenamiento. Por defecto, tendrás una <font color="orange">**Tesla T4**</font>.

In [ ]:
#@markdown # <font color="pink"> **Montar Google Drive.** 📂
#@markdown ---
#@markdown **Se necesita como mínimo <font color="orange">2 GB</font> de espacio libre.**
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
#@markdown # <font color="pink"> **Instalar software.** 📦
#@markdown ---
#@markdown En esta celda se instalará todo lo necesario para el funcionamiento del cuaderno.

#@markdown <font color="orange">**Nota importante: por favor, reinicia el entorno de ejecución cuando termine esta celda de ejecutarse. A continuación, ejecuta nuevamente esta celda y sigue tu camino. (Sea entrenamiento o inferencia.)**
#@markdown #### Versión a usar:
#@markdown **(Se recomienda 4.0 (mejorado) para compatibilidad al cargar modelos para [FakeYou](https://fakeyou.com/) o [Uberduck](https://uberduck.ai/)). 4.0-v2 para resultados ligeramente mejores en la inferencia. Sin embargo, no es compatible con estas webs ya mencionadas.**
%cd /content
import os
version = "4.0 (mejorado)" #@param ["4.0", "4.0 (mejorado)", "4.0-v2"]
if not os.path.exists("/content/so-vits-svc"):
    if version == "4.0 (mejorado)":
        !python -m pip install -U pip wheel
        !pip install -U git+https://github.com/34j/so-vits-svc-fork@9c8df717ae433fe3f3e2ef6ee49a1a165c66cf03
    else:
        !git clone https://github.com/rmcpantoja/so-vits-svc -b {version}
if os.path.exists("/content/so-vits-svc"):
    %cd /content/so-vits-svc
    !pip install --upgrade pip setuptools numpy numba gdown
    !pip install pyworld praat-parselmouth fairseq tensorboardX librosa==0.9.1
    !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117
if not version == "4.0 (mejorado)":
    if not os.path.exists("/content/so-vits-svc/hubert/checkpoint_best_legacy_500.pt"):
        if version == "4.0":
            !wget -P hubert/ https://huggingface.co/justinjohn-03/so-vits-svc-4.0/resolve/main/checkpoint_best_legacy_500.pt
        else:
            !wget -P hubert/ https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/checkpoint_best_legacy_500.pt
else:
  print("Descargando parches...")
  !gdown "1-3MXtoGQiIjTSobFXEcwRIFu8qB6Ya-v" -O "/usr/local/lib/python3.10/dist-packages/so_vits_svc_fork/train.py"

# <font color="pink"> 🤖 ***Entrenamiento.*** 🤖

In [ ]:
#@markdown # <font color="pink"> **Configuración.** 🧰
if not version == "4.0 (mejorado)":
    %cd /content/so-vits-svc
else:
    !mkdir -p "dataset_raw"
import os

#@markdown ---
#@markdown #### Procesamiento de voz
#@markdown <font color="orange">**Método que se va a usar:**
mode = "una sola voz" #@param ["una sola voz", "varias voces"]
#@markdown <font color="orange">**¿Ruta de los audios?**</font>
#@markdown Esta tendrá que estar comprimida en un archivo zip. Sin embargo, algunos puntos a tomar en cuenta:
#@markdown * Los audios de la voz deberán estar recortados de entre 9-15 segundos.
#@markdown * Si seleccionas el método para varias voces, asegúrate de que el zip tenga una carpeta por voz, además de que el nombre del cantante se tomará en cuenta solamente para la ruta de guardado, pero el modelo puede tener tantas voces con los nombres diferentes mientras sea posible, así que puedes elegir un nombre claro y conciso. Ejemplo: "Grupo 5"
#@markdown * Caso contrario, pon el nombre del cantante en el campo respectivo, ya que también es importante para establecerlo como único hablante (en la configuración del modelo) y como nombre de carpeta en el directorio de trabajo.
zip_path = "/content/drive/MyDrive/wavs.zip" #@param {type:"string"}
#@markdown **Nombre del cantante:**
voicename = "Cantante" #@param {type:"string"}
if not os.path.exists(zip_path):
  raise Exception("¡Este archivo zip no existe! Por favor, comprueba que no hay errores tipográficos al establecer la ruta.")
print("Extrayendo audio...")
if mode == "una sola voz":
  if not version == "4.0 (mejorado)":
    folder = "/content/so-vits-svc/dataset_raw/"+voicename
  else:
    folder = "/content/dataset_raw/"+voicename
  !mkdir "{folder}"
  !unzip -q "{zip_path}" -d "{folder}"
else:
  if not version == "4.0 (mejorado)":
    !unzip -q "{zip_path}" -d /content/so-vits-svc/dataset_raw
  else:
    !unzip -q "{zip_path}" -d /content/dataset_raw
#@markdown **Remuestrar audios**

#@markdown El formato compatible para el entrenamiento es de <font color="orange">**44khz, 16bits, mono.**</font> Si no tienes los audios con este formato, habilita la casilla para hacer la conversión.
resample = True #@param {type:"boolean"}
if resample:
  print("Remuestrando audios...")
  if not version == "4.0 (mejorado)":
    !python resample.py
  else:
    !svc pre-resample
else:
  if not version == "4.0 (mejorado)":
    !mv /content/so-vits-svc/dataset_raw/* /content/so-vits-svc/dataset
#@markdown #### **Guardado**
#@markdown <font color="orange">**¿Dónde guardar el trabajo?**</font>
#@markdown Se recomienda guardarlo en drive.
save_dir = "/content/drive/MyDrive/SoVits" #@param {type:"string"}
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
if version == "4.0 (mejorado)":
    if not os.path.exists(save_dir+"/"+voicename):
        os.makedirs(save_dir+"/"+voicename)
#@markdown #### **¿Usar el modelo preentrenado?** (Obligatorio)
#@markdown Activa esta casilla si no tienes suficientes horas de audio. Siendo así, se recomiendan <font color="orange">**15 minutos como mínimo.**<br>***ESTA CASILLA DEBES DESACTIVARLA SI VAS A SEGUIR UN ENTRENAMIENTO.***</font>
pre_pth = True #@param {type:"boolean"}
if pre_pth:
  if not version == "4.0 (mejorado)":
    if not os.path.exists(save_dir+"/"+voicename):
      print("Descargando modelo preentrenado...")
      if version == "4.0":
        !wget -P {save_dir}/{voicename}/ https://huggingface.co/justinjohn-03/so-vits-svc-4.0/resolve/main/G_0.pth
        !wget -P {save_dir}/{voicename}/ https://huggingface.co/justinjohn-03/so-vits-svc-4.0/resolve/main/D_0.pth
      else:
        !wget -P {save_dir}/{voicename}/ https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/G_0.pth
        !wget -P {save_dir}/{voicename}/ https://huggingface.co/justinjohn-03/so-vits-svc-4.0-v2-pretrained/resolve/main/D_0.pth
    else:
      print("¡Aviso! Tienes activada la opción de usar modelo preentrenado, pero ¡hemos detectado que este modelo ya se ha entrenado anteriormente! Por lo tanto, asumiremos que lo quieres continuar. No se descargará.")
#@markdown **Selecciona el método de extracción de tono (F_O):**

#@markdown  <font color="orange">**Nota: esta opción tiene efecto solamente para la versión 4.0 mejorada.**
F0_METHOD = "dio" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]

print("Procesando archivos...")
if not version == "4.0 (mejorado)":
  !python preprocess_flist_config.py
  !python preprocess_hubert_f0.py
else:
  !svc pre-config
  !cp "configs/44k/config.json" "{save_dir}/{voicename}"
  !svc pre-hubert -fm {F0_METHOD}

In [ ]:
#@markdown # <font color="pink"> **Ejecutar la extensión TensorBoard.** 📈
#@markdown ---
#@markdown Aquí puedes visualizar los datos del entrenamiento, tales como audios y gráficas.
%load_ext tensorboard
%tensorboard --logdir {save_dir}

In [ ]:
#@markdown # <font color="pink"> **Comenzar el entrenamiento.** 🏋️‍♂️
#@markdown ---
#@markdown Se recomienda entrenar un mínimo de <font color="orange">**5.000 pasos**</font> para que sea decente.
if not version == "4.0 (mejorado)":
  !python train.py -c configs/config.json -m {voicename} -p {save_dir}
else:
  !svc train --model-path "{save_dir}/{voicename}"

# <font color="pink"> 🗣️ ***Inferencia.*** 🗣️

### Asumiendo que ejecutas el apartado de <font color="orange">**preparación del proyecto**</font>, puedes proceder a ejecutar esta sección para probar un modelo previamente entrenado.

In [ ]:
#@markdown # <font color="pink"> **Gestionar modelos.** 📊
#@markdown ---
#@markdown Si quieres descargar un modelo nuevo o cambiar otro, puedes hacerlo desde aquí:

#@markdown <font color="orange">**ID o enlace de Google Drive</font> del archivo del modelo a descargar:**
model_id = "" #@param {type:"string"}
#@markdown <font color="orange">**ID o enlace de Google Drive</font> del archivo de configuración (config.json):**
config_id = "" #@param {type:"string"}
if version == "4.0 (mejorado)":
  !mkdir /content/so-vits-svc
  !mkdir /content/so-vits-svc/logs
  !mkdir /content/so-vits-svc/logs/44k
  !mkdir /content/so-vits-svc/raw
  !mkdir /content/so-vits-svc/results
%cd /content/so-vits-svc
#descarga los archivos:
if model_id.startswith("1"):
  !gdown "{model_id}" -O "/content/so-vits-svc/logs/44k/pretrained.pth"
elif model_id.startswith("https://drive.google.com/file/d/"):
  !gdown "{model_id}" -O "/content/so-vits-svc/logs/44k/pretrained.pth" --fuzzy
else:
  !wget "{model_id}" -O "/content/so-vits-svc/logs/44k/pretrained.pth"
if config_id.startswith("1"):
  !gdown "{config_id}" -O "/content/so-vits-svc/logs/44k/config.json"
elif config_id.startswith("https://drive.google.com/file/d/"):
  !gdown "{config_id}" -O "/content/so-vits-svc/logs/44k/config.json" --fuzzy
else:
  !wget "{config_id}" -O "/content/so-vits-svc/logs/44k/config.json"

In [ ]:
#@markdown # <font color="pink"> **Ejecutar la inferencia.** 🔊
from IPython.display import Audio, display
import os
#@markdown **Selecciona el modo de subida del audio:**

#@markdown Para el modo manual, sube tu audio desde el panel izquierdo <font color="orange">**(carpeta So-Vits-SVC/raw)**</font> y pon el nombre del audio que has subido en el campo de edición; o, de lo contrario, sube tu audio en el panel que se te mostrará al ejecutar la celda:

upload_mode = "Manual" #@param ["Automático", "Manual"]
if upload_mode == "Automático":
  %cd /content/so-vits-svc/raw
  from google.colab import files
  wav_filename, length = files.upload().popitem()
  %cd ..
else:
  #@markdown **¿Cuál es el nombre del archivo de audio a convertir?**
  wav_filename = "miaudio.wav" #@param {type:"string"}
audio_ruta = "/content/so-vits-svc/raw/"+wav_filename
if not os.path.exists(audio_ruta):
  raise Exception(f"El audio especificado no existe. Por favor, verifica que este existe: {audio_ruta}")
#@markdown **Nombre del hablante de los establecidos previamente en la configuración de tu modelo:**
speaker = "Cantante" #@param {type:"string"}
#@markdown **Tono de la voz. Puedes jugar con esto según el rango vocal de la voz entrenada. Ten en cuenta que valores mayores o menores al rango puede sonar forzado.**
trans = 0 #@param {type:"integer"}
cluster_infer_ratio = "0"
config_path = "/content/so-vits-svc/logs/44k/config.json"
if not os.path.exists(config_path):
  raise Exception("El archivo de configuración no existe. Asumo que falló al descargar. En todo caso, ejecuta la celda de gestionar modelos para verificar si hay errores.")
model_path = "/content/so-vits-svc/logs/44k/pretrained.pth"
if not os.path.exists(model_path):
  raise Exception("Ups, parece que el modelo falló al descargar. Ejecuta nuevamente la celda de gestión de modelos y comprueba si hay errores.")
slice_db = "-40"
#@markdown **Elige el formato de audio de exportación:**
wav_format = "wav"  #@param ["flac", "ogg", "wav"]
wav_output = "/content/so-vits-svc/results/" + wav_filename + "_" + trans + "key" + "_" + speaker + "." + wav_format

#@markdown **Selecciona el método de extracción de tono:**
F0_METHOD = "dio" #@param ["crepe", "crepe-tiny", "parselmouth", "dio", "harvest"]
#@markdown **¿Activar predicción F_0?**<br>
#@markdown <font color="orange">**Solo sirve en los modelos 4.0 mejorados.**

#@markdown Activado para habla, desactivado para canto.
auto_f0 = False #@param {type:"boolean"}
if auto_f0:
  autof0 = "-a"
else:
  autof0 = "-na"
#@markdown <font color="orange">**Nota: si al ejecutar la celda obtienes el error llamado: "can't convert string to float", asegúrate de haber reiniciado el entorno de ejecución, ejecutar nuevamente la celda de instalación de software e inferencia.**
if not version == "4.0 (mejorado)":
  !python inference_main.py -n {wav_filename} -m {model_path} -c {config_path} -s {speaker} -t {trans} -cr {cluster_infer_ratio} -sd {slice_db} -wf {wav_format}
else:
  !svc infer {audio_ruta} -m {model_path} -c {config_path} -s {speaker} -t {trans} -r {cluster_infer_ratio} -db {slice_db} -o "{wav_output}" -fm {F0_METHOD} {autof0}
Audio(wav_output, rate=44100)

# <font color="pink"> 📓 ***Cuadernos relacionados o de interés.*** 📓

* [VITS: Codificador automático variacional condicional con aprendizaje adversario para texto a voz de extremo a extremo. (en español, beta)](https://colab.research.google.com/github/rmcpantoja/My-Colab-Notebooks/blob/main/notebooks/cuaderno_de_entrenamiento_VITS_en_espa%C3%B1ol.ipynb)